#### Change the promting according to the promting of pip-sql-1.3b
https://huggingface.co/PipableAI/pip-sql-1.3b

#### Go from using langchain sql chain to manual promt template.

In [1]:
from sqlalchemy import create_engine, text, MetaData
from langchain.sql_database import SQLDatabase

db_files = '/home/arseniy/Документы/Projects/Python/LLM_project/db_files/'

engine = create_engine ('sqlite:///' + db_files + 'mys_db', echo= False, future=True)

metadata = MetaData()
metadata.reflect(bind=engine)

db = SQLDatabase(engine)


In [2]:
from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(
    public_key="key",
    secret_key="key",
    host="https://cloud.langfuse.com"
)

In [3]:
from langchain_community.chat_models import ChatLlamaCpp 
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
import multiprocessing
from langchain.globals import set_llm_cache
# Path to the model weights
models_directory = '/home/arseniy/Документы/Projects/Python/LLM_project/models/'
'''
small_model models_directory + 'qwen2-0_5b-instruct-q8_0.gguf'
sql_model = models_directory + 'sqlcoder-7b-2.Q4_K_M.gguf' 
'''
# First try using same big LLM in twice and keep it in memory.
local_model = models_directory + 'Mistral-7B-Instruct-v0.3.Q5_K_M.gguf' 

llm = ChatLlamaCpp(
    model_path = local_model,
    n_ctx=2000,
    n_gpu_layers=10,
    use_mlock = True,
    versbose = True,
)

'''
table_name_extractor = ChatLlamaCpp(
    cache = True,
    model_path = local_model,
    n_ctx=2000,
    n_gpu_layers=10,
    use_mlock = True,
    versbose = True,
    
)

query_generator = ChatLlamaCpp(
    cache = True,
    model_path = local_model,
    n_ctx=2000,
    n_gpu_layers=10,
    use_mlock = True,
    versbose = True,
    
)
'''

llama_model_loader: loaded meta data with 29 key-value pairs and 291 tensors from /home/arseniy/Документы/Projects/Python/LLM_project/models/Mistral-7B-Instruct-v0.3.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models--mistralai--Mistral-7B-Instruc...
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 32768
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32  

'\ntable_name_extractor = ChatLlamaCpp(\n    cache = True,\n    model_path = local_model,\n    n_ctx=2000,\n    n_gpu_layers=10,\n    use_mlock = True,\n    versbose = True,\n    \n)\n\nquery_generator = ChatLlamaCpp(\n    cache = True,\n    model_path = local_model,\n    n_ctx=2000,\n    n_gpu_layers=10,\n    use_mlock = True,\n    versbose = True,\n    \n)\n'

In [4]:
from operator import itemgetter
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chains import LLMChain

In [5]:
with open(db_files + 'raw.txt', 'r') as file:
    schema = file.read()


In [6]:
question = ""

template = """
Given the following database schema:

{schema}

Please identify the names of the relevant tables that can be used to query the database based on the following user question:

"{prompt}"

Respond with only the names of the relevant tables.Do not write anything else. If the prompt involves multiple tables, list all the relevant table names separated by commas. List all related table names even if you thin that they are not so related to the prompt.

"""
# Initialize the prompt template with placeholders for 'schema' and 'question'
prompt_template = PromptTemplate.from_template(template)

In [7]:
chain = LLMChain(prompt=prompt_template, llm=llm)

/home/arseniy/.var/app/org.jupyter.JupyterLab/config/jupyterlab-desktop/jlab_server/envs/llamaCuda/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [8]:
question = input("🟢 Enter your question: ")
response = chain.run(prompt = question, schema=schema)
print(response)

🟢 Enter your question:  where is employee with id 3 living


/home/arseniy/.var/app/org.jupyter.JupyterLab/config/jupyterlab-desktop/jlab_server/envs/llamaCuda/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(

llama_print_timings:        load time =     831.56 ms
llama_print_timings:      sample time =       4.68 ms /    10 runs   (    0.47 ms per token,  2135.84 tokens per second)
llama_print_timings: prompt eval time =  189559.88 ms /  1644 tokens (  115.30 ms per token,     8.67 tokens per second)
llama_print_timings:        eval time =    1983.16 ms /     9 runs   (  220.35 ms per token,     4.54 tokens per second)
llama_print_timings:       total time =  192243.29 ms /  1653 tokens


 employees, orgHierarchy, workHistory


In [13]:
def parse_words(input_string):
    # Remove leading and trailing spaces, then split by commas
    words = input_string.strip().split(',')
    
    # Strip any leading/trailing spaces from each word
    words = [word.strip() for word in words]
    
    return words

table_schemas = parse_words(response)
print(table_schemas)

['employees', 'orgHierarchy', 'workHistory']


In [14]:
template1 =  """
                Generate a SQL query to answer the user question: "{question}".
                
                The query will run on a database with the following schema:
                {table_schemas}
                
                Write only the query, nothing more!
            """
prompt_template1 = PromptTemplate.from_template(template1)

In [15]:
chain1 = LLMChain(prompt=prompt_template1, llm=llm)

In [18]:

response1= chain1.run(question = question, table_schemas=table_schemas)
print(response1)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     831.56 ms
llama_print_timings:      sample time =      16.68 ms /    39 runs   (    0.43 ms per token,  2338.41 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =    7869.08 ms /    39 runs   (  201.77 ms per token,     4.96 tokens per second)
llama_print_timings:       total time =    7904.10 ms /    39 tokens


 SELECT `address` FROM `employees` WHERE `id` = 3;

This query will return the address of employee with id 3 from the employees table.


In [ ]:
"""
import sqlvalidator
import sqlcorrect as sc

def main():
    print("🔴 Simple database chatbot app. Type 'exit' to quit.")
    while True:
        # Get user input
        question = input("🟢 Enter your question: ")
        if question.lower() == 'exit':
            print("Exiting the app.")
            break 

        response = chain.run(question = question, schema=description)
        try:            
            print("Response:", response) 
        except:
            print("🔴 LLM agent gave a wrong SQL query. Paraphrase the question...")
        

if __name__ == "__main__":
    main()
"""

#### Classify related table names with smaller model -> query the database with related names -> pass table schemas to the sqlcoder.
#### Think about using cache, especially redis db as cache.